In [23]:
import SatelliteNavigation as SN
using SatelliteDynamics
using Distributions

In [24]:
#Doppler Custom Units
distance_scale = R_EARTH*(1e-3) # scales km to custom scale working

time_scale = (1/(C_LIGHT/R_EARTH))*1e4 #scales seconds to custom scale

velocity_scale = distance_scale/time_scale #scales velocities to custom scales

frequency_scale = 1

c = 1*1e4 #km/s

10000.0

In [25]:
#Set the tag position

#tag position in geodetic coordinates
tag_geof = [-165.4545, 0, 0]

#tag position in ECEF frame
tag = sGEOCtoECEF(tag_geof, use_degrees = true) #in meters

3-element Vector{Float64}:
 -6.173708155673107e6
 -1.6018611673126891e6
  0.0

In [26]:
#Define Orbit Parameters

trueanom = 10
RAAN_sep = 2
delta_sep = 3
altitude = 500
satnum = 4

orbit_params = [trueanom, RAAN_sep, delta_sep, altitude, satnum]

#obtain the state of each satellite
combined_eci = SN.create_sat_orbit(orbit_params);

In [27]:
#visualize the constellation
SN.plot_sat_constellation(combined_eci, tag, satnum)

data: [
  "scatter3d with fields mode, name, type, x, y, and z",
  "scatter3d with fields mode, name, type, x, y, and z",
  "scatter3d with fields marker, mode, name, type, x, y, and z",
  "scatter3d with fields marker, mode, name, type, x, y, and z",
  "surface with fields showscale, type, x, y, and z"
]

layout: "layout with fields margin and template"

In [28]:
#Generate TEC Distribution
mu = 8e-4
sigma = 3e-4
lb = 3e-4
ub = 13e-4
d = Truncated(Normal(mu,sigma), lb, ub)

#Estimated ionosphere as a thin shell. hi is the height of the shell
hi = 350e3 #in meters

#transmit at how many frequencies. If only using 1 frequency, set f2=0
f1 = 400e6
f2 = 600e6
frequency = [f1, f2]

2-element Vector{Float64}:
 4.0e8
 6.0e8

In [29]:
#set r0 scaled
bdot = 1e4*1e-3 
r0_scaled = [tag*1e-3/distance_scale; bdot/velocity_scale]

4-element Vector{Float64}:
 -0.9679486083847263
 -0.2511487826487322
  0.0
  0.33356409519815206

In [30]:
n=1000
PDOP_array = zeros(size(combined_eci)[2],1)

for i=1:size(combined_eci)[2]
    
    satposes = [combined_eci[1,i] combined_eci[2,i] combined_eci[3,i];combined_eci[7,i] combined_eci[8,i] combined_eci[9,i]; combined_eci[13,i] combined_eci[14,i] combined_eci[15,i]; combined_eci[19,i] combined_eci[20,i] combined_eci[21,i]]

    zenith_angles = SN.zenith(satposes, tag, satnum)
    
    #add in with for loop
    inhorizon = all(x->x<70, zenith_angles)
    
    if inhorizon==false
        continue #skip the current iteration
    end
        
    mean_rescaled, all_r0, iters = SN.get_mean_Doppler(combined_eci, i, d, r0_scaled, frequency, zenith_angles, satnum, tag)
    
    PDOP = SN.find_covariance(all_r0, mean_rescaled)
    
    PDOP_array[i,1] = PDOP
    
    println("Mean: ", mean_rescaled)
    println("Pose Standard Dev: ", PDOP)
    println("Step: ", i)
    
end

Mean: [-6.174016906268869e6, -1.6014506109855517e6, 275.11948088470416, 10000.968322904304, 0.001154448168511696, -1.10478358685043e-12, 0.0007413663136016143, 0.0006808128958462484]
Pose Standard Dev: 1551.5715596638402
Step: 152
Mean: [-6.174077365314533e6, -1.6013377984893587e6, 326.8291962218226, 10001.12403859955, 0.001027946941763662, -1.1450160216212429e-12, 0.0012029548064846113, 0.0006384009392637417]
Pose Standard Dev: 1550.249793632642
Step: 153
Mean: [-6.1740752879129145e6, -1.6013568991503045e6, 340.4677859191607, 10001.15433678637, 0.0006739454239223458, -1.143654017192116e-12, 0.00034687575346691496, 0.0008605242794862651]
Pose Standard Dev: 1586.2335148489747
Step: 154
Mean: [-6.174156618834633e6, -1.601224270795201e6, 407.217211173719, 10001.396145251552, 0.0010010749095731664, -1.1754204894468402e-12, 0.0009139602278809612, 0.00040390665287350563]
Pose Standard Dev: 1640.096925624409
Step: 155
Mean: [-6.174330792230636e6, -1.600962790627587e6, 579.884968477477, 10001.

Mean: [-6.182348217341162e6, -1.5781786740364376e6, 13408.368102567538, 10044.066660672333, 0.000542156791120008, -1.8063142271397426e-12, 0.0006832902502222816, 0.0011483753625405931]
Pose Standard Dev: 7936.4670584224505
Step: 188
Mean: [-6.18162549416436e6, -1.5795898248491478e6, 12472.648915066702, 10041.045854861688, 0.0008834262195465151, -1.3762773547279987e-12, 0.0007386141418451146, 0.0009635056434854352]
Pose Standard Dev: 9612.795998270592
Step: 189
Mean: [-6.185121006395618e6, -1.5700656151611279e6, 17673.60429377431, 10058.208617110298, 0.0010777693221483643, -1.6062003024082702e-12, 0.0006152442714322095, 0.00036671101967874624]
Pose Standard Dev: 11353.70874847107
Step: 190
Mean: [-6.187305668039055e6, -1.5639617411139645e6, 20856.124218894132, 10068.844538301546, 0.001202731401689593, -1.450080804584128e-12, 0.000731459592010312, 0.0007500093909889893]
Pose Standard Dev: 13875.527063881385
Step: 191
Mean: [-6.199131738590014e6, -1.5365725891981302e6, 35906.93575555719, 

Mean: [-6.172578270040008e6, -1.6125470884609327e6, -3120.1254196091495, 9987.753801568582, 0.0009884990434264366, -1.3982501668715925e-12, 0.0004666047582480218, 0.0007089750952971532]
Pose Standard Dev: 1924.8851912431053
Step: 224
Mean: [-6.172556622918159e6, -1.613212266650042e6, -3223.3326323388146, 9987.273321848927, 0.00042588170296798956, -1.4813609626833035e-12, 0.0008262777152707886, 0.0008350371594372076]
Pose Standard Dev: 1916.1731853752708
Step: 225
Mean: [-6.172597656792926e6, -1.6129639978490626e6, -3063.8360628768473, 9987.74964949352, 0.00042289568480250025, -1.4697047996041919e-12, 0.001009698178387734, 0.0007018979938575115]
Pose Standard Dev: 1799.69470228897
Step: 226
Mean: [-6.172054150889091e6, -1.6197094883626162e6, -4763.313263686835, 9980.657322339393, 0.0007174062322810325, -2.418653607185956e-12, 0.0006024955951016438, 0.0009684855977240343]
Pose Standard Dev: 1742.7291618042018
Step: 227
Mean: [-6.17223009216203e6, -1.6179015906075824e6, -4140.784083479704

In [31]:
final_PDOP = PDOP_array[PDOP_array .!=0]

PDOP_max = findmax(final_PDOP)[1]
PDOP_min = findmin(final_PDOP)[1]

println("Maximum PDOP: ", PDOP_max)
println("Minimum PDOP: ", PDOP_min)

Maximum PDOP: 5.934232202752279e10
Minimum PDOP: 1373.4096660234366
